In [1]:
import pandas as pd
import numpy as np
import json
from scipy import sparse as sp
from tqdm.notebook import tqdm
from collections import defaultdict

In [2]:
import sys
sys.path.append('../')

from src.utils import get_shard_path
from src.metrics import normalized_average_precision

baseline - просто посчитать самый популярный товар и рекомендовать его всем, независимо от пользователя:

In [5]:
cnt = defaultdict(int)

def read_part(path):
    for js in tqdm((json.loads(s) for s in open(path))):
        for trans in js['transaction_history']:
            for product in trans["products"]:
                cnt[product["product_id"]] += 1

for i in range(4):
    read_part(get_shard_path(i))

_tmp = list(cnt.keys())
top_products = sorted(_tmp, key=lambda x: -cnt[x])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
top_products

['4009f09b04',
 '15ccaa8685',
 'bf07df54e1',
 '3e038662c0',
 '4dcf79043e',
 'f4599ca21a',
 '5cb93c9bc5',
 '4a29330c8d',
 '439498bce2',
 '343e841aaa',
 '0a46068efc',
 'dc2001d036',
 '31dcf71bbd',
 '113e3ace79',
 '5645789fdf',
 'f098ee2a85',
 '080ace8748',
 '53fc95e177',
 '4c07cb5835',
 'ea27d5dc75',
 'cbe1cd3bb3',
 '1c257c1a1b',
 'f5e18af323',
 '5186e12ff4',
 '6d0f84a0ac',
 'f95785964a',
 'de25bccdaf',
 'ac81544ebc',
 'ad865591c6',
 'f43c12d228',
 'd860464ae1',
 'fc5b0d84e8',
 'e6f8ac5174',
 '9b8d3ecb9c',
 'f7724d5786',
 '67e5809518',
 '76ae00433f',
 'a02d521260',
 'c5e3528190',
 '4bd01c5234',
 '0cbc352942',
 '4d3ab3e72c',
 '298c1acea3',
 '719b704cb6',
 '19ae13ddb0',
 'c85e209e7d',
 '86ffd3a584',
 '341735fe50',
 '8d4f7352d7',
 '70b7a4e9da',
 '88ac98dea5',
 'ee17733c1a',
 'b43b19cea8',
 'e6a5597d19',
 '647c87e132',
 'ed53a4b7bd',
 'f587bd2528',
 'ccc2662707',
 '262509c05d',
 '5ae28e9988',
 '7c39f1d12c',
 'd16a7d2bfc',
 '8f45a9ec73',
 '70d4730470',
 '5c6de3010f',
 '3a8eeabcab',
 '9bc0a4e8

In [9]:
print(cnt['4009f09b04'])
print(cnt['15ccaa8685'])

766595
266330


# Check MAP@30

In [10]:
scores = []
for js in tqdm((json.loads(s) for s in open(get_shard_path(7)))):
    recommended_items = top_products[:30]
    gt_items = js["target"][0]["product_ids"]
    ap = normalized_average_precision(gt_items, recommended_items)
    scores.append(ap)
np.mean(scores)

0it [00:00, ?it/s]

0.06490413817015536